# 더미변수

명목형 변수에 대한 표준화 처리 방법 (명목형 변수를 연속형변수스럽게 만드는 것)

`patsy` 패키지의 설치가 필요하다

#### 왜 더미변수를 만드는가? 

범주형 변수로는 사용할 수 없고 연속형변수로만 가능한 분석기법을 사용할 수 있다. (선형회귀분석, 로지스틱 회귀분석 등등...)

#### 더미변수의 의미

더미변수는 회귀식에서 해당 변수의 효과를 0 또는 상수값으로 만들어준다. 



In [21]:
from pandas import read_excel, DataFrame
from patsy import dmatrix
import numpy as np

In [5]:
df = read_excel("https://data.hossam.kr/C02/dum.xlsx")
df

,성별,비만도
0,남자,정상
1,여자,경도
2,여자,정상
3,남자,고도
4,남자,정상
5,남자,경도
6,남자,고도
7,여자,고도
8,여자,경도
9,남자,고도


### 명목형 변수 라벨링

남자:0 / 여자:1

컬럼이름을 

In [9]:
dmatrix('성별+0', df)

DesignMatrix with shape (20, 2)
  성별[남자]  성별[여자]
       1       0
       0       1
       0       1
       1       0
       1       0
       1       0
       1       0
       0       1
       0       1
       1       0
       0       1
       0       1
       0       1
       1       0
       0       1
       0       1
       1       0
       0       1
       0       1
       0       1
  Terms:
    '성별' (columns 0:2)

In [8]:
dmatrix('비만도+0', df)

DesignMatrix with shape (20, 3)
  비만도[경도]  비만도[고도]  비만도[정상]
        0        0        1
        1        0        0
        0        0        1
        0        1        0
        0        0        1
        1        0        0
        0        1        0
        0        1        0
        1        0        0
        0        1        0
        0        0        1
        0        0        1
        0        0        1
        0        0        1
        0        1        0
        1        0        0
        0        1        0
        1        0        0
        0        0        1
        1        0        0
  Terms:
    '비만도' (columns 0:3)

In [12]:
# 명목형 변수 2개
dm = dmatrix('성별: 비만도 + 0', df)
dm

DesignMatrix with shape (20, 6)
  Columns:
    ['성별[남자]:비만도[경도]',
     '성별[여자]:비만도[경도]',
     '성별[남자]:비만도[고도]',
     '성별[여자]:비만도[고도]',
     '성별[남자]:비만도[정상]',
     '성별[여자]:비만도[정상]']
  Terms:
    '성별:비만도' (columns 0:6)
  (to view full data, use np.asarray(this_obj))

In [14]:
dmarray = np.asarray(dm)
dmarray
# [0., 0., 0., 0., 1., 0.] = 남자경도 / 여자경도 / 남자고도/ 여자고도/ 남자정상(1)/ 여자정상

array([[0., 0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1.],
       [0., 1., 0., 0., 0., 0.]])

In [ ]:
dummy_df = DataFrame(dmarray, columns=dm.desgin_info.column_names)
dummy_df

## 원본데이터가 라벨링 되어 있는 경우

In [16]:
df2 = df.copy()
df2.head()

,성별,비만도
0,남자,정상
1,여자,경도
2,여자,정상
3,남자,고도
4,남자,정상


In [17]:
# 컬럼을 구분하지 않고 모든 값을 변경
df2.replace('남자', 0, inplace=True)
df2.replace('여자', 1, inplace=True)

# 성별 컬럼에서만 변경
df2.replace({'성별':'정상'}, 0, inplace=True)
df2.replace({'성별':'경도'}, 1, inplace=True)
df2.replace({'성별':'고도'}, 2, inplace=True)
df2.head()

,성별,비만도
0,0,0
1,1,1
2,1,0
3,0,2
4,0,0


#### 라벨링 된 데이터의 더미 변수화

표현식에 범주형(Category)임을 의미하는 `C`를 표기

In [ ]:
df